# Parsing basics

## Objectives

*   write a function that will be based on a site search
*   based on the materials included in the search results for all keywords, return a dataframe


In [23]:
#import all required libraries
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [24]:
#make a list of words for search
query = ['Java', 'python']

In [25]:
#this function will take a list of words as a parameter and return a list of the necessary links
def get_all_links(query):
    URL = 'https://habr.com/ru/search/'
    all_refs = []
    for word in query:
        params = {
            'q': word,
        }
    res = requests.get(URL, params)
    time.sleep(0.3) 
    soup = BeautifulSoup(res.text, 'html.parser')
    # find all news blocks:
    news_blocks = soup.find_all('div', class_='tm-article-snippet')
    # find the text with the link
    articles_intro = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'), news_blocks))
    # find all links
    a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
    # create full-fledged links
    all_refs += list(map(lambda x: 'https://www.habr.com' + x, a_list))
    return all_refs

all_links = get_all_links(query)
len(all_links)

20

In [26]:
# this function will take a list of links and return a pandas dataframe with the form: date, title, link to the material
def get_habr_news(links):
    habr_news = pd.DataFrame()
    for link in all_links:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        # find a news's date
        date = pd.to_datetime(soup.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True).date()
        # find a news's title
        title = soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1').text       
        row = {'DATE': date, 'TITLE': title, 'LINK': link}
        habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
    return habr_news.drop_duplicates()

In [27]:
habr_news = get_habr_news(all_links)
habr_news

,DATE,TITLE,LINK
0,2021-12-13,Жаждущим автоматизации: открытый урок «ChatOps...,https://www.habr.com/ru/company/southbridge/ne...
0,2022-01-20,Курс «Python для инженеров». Старт 3 потока 31...,https://www.habr.com/ru/company/southbridge/ne...
0,2020-04-21,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://www.habr.com/ru/news/t/498364/
0,2021-07-06,Python Community Meetup 8/07: видео и материал...,https://www.habr.com/ru/company/raiffeisenbank...
0,2022-01-13,Открытый урок «Пишем Custom Prometheus Exporte...,https://www.habr.com/ru/company/southbridge/ne...
0,2020-12-04,Python как компилируемый статически типизирова...,https://www.habr.com/ru/news/t/531402/
0,2022-03-08,Вышел мартовский релиз расширения Python для V...,https://www.habr.com/ru/news/t/654707/
0,2020-03-03,В начале этого года Python сместил Java и стал...,https://www.habr.com/ru/company/itsumma/news/t...
0,2021-11-16,EPAM разработала бесплатный курс по программир...,https://www.habr.com/ru/company/epam_systems/n...
0,2020-06-03,Вышла версия 0.0.2 snakeware — дистрибутива Li...,https://www.habr.com/ru/news/t/505096/
